In [14]:
import re
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
import csv
import re


In [15]:
tuyendung = pd.read_csv("output1.csv")
tuyendung.head()

,post_id,content,images
0,2861071620726465,🔥 𝐍𝐚𝐧𝐨𝐛𝐲𝐭𝐞 𝐓𝐮𝐲𝐞̂̉𝐧 𝐃𝐮̣𝐧𝐠 REMOTE ANDROID Develo...,[]
1,2860805674086393,Công ty IT vừa thành lập nửa năm cần tuyển key...,[]
2,2860731854093775,"Hôm nay trời nắng, cty mình cần gấp 2 bạn fron...",[]
3,2860903027409991,[PICON TECHNOLOGY - CHÚNG MÌNH TÌM UX/UI DESIG...,[]
4,2861061440727483,ĐN- bên em dự án lớn tuyển dev C/C++Software E...,[]


In [16]:
content = tuyendung['content']
content.head(40)

0     🔥 𝐍𝐚𝐧𝐨𝐛𝐲𝐭𝐞 𝐓𝐮𝐲𝐞̂̉𝐧 𝐃𝐮̣𝐧𝐠 REMOTE ANDROID Develo...
1     Công ty IT vừa thành lập nửa năm cần tuyển key...
2     Hôm nay trời nắng, cty mình cần gấp 2 bạn fron...
3     [PICON TECHNOLOGY - CHÚNG MÌNH TÌM UX/UI DESIG...
4     ĐN- bên em dự án lớn tuyển dev C/C++Software E...
5     (TP Huế) Tuyển\nC/C++ or Python or Swift dev ~...
6     Đà Nẵng tuyển\nIT COMTOR\nTiếng Nhật N1, CÓ kn...
7     1. Job Remote Fulltime - Senior Front-End Deve...
8     Đà Nẵng tìm gấp\nPHP/ C++ WITH GOOD ENG\nlevel...
9     [AVEPOINT – DANANG]\n“Dính cứng ngắc” với hệ t...
10    Mục tiêu R500 sẵn sàng, welcome 500 ae về Rikk...
11    Ở đây chúng mình có "Evizi, Job xịn và Lương c...
12    💥CÔNG TY ZIMAW - TUYỂN DỤNG NHỀU VỊ TRÍ💥💥\nLIN...
13    ĐÀ NẴNG OPEN JOB PHP/ JAVA/ TESTER/ REACTJS 🤤🤤...
14    Mình cần Xây Đội DEV lập trình back-end , chuy...
15    TÌM KIẾM 50 NHÂN SỰ MỚI OB TRONG T3 NÀY\nVới m...
16    Các anh chị nhân sự ơi, có thực sự cần tuyển d...
17    Để có thêm " kiến thức, trải nghiệm, kinh 

# Vẽ biểu đồ ngôn ngữ thông dụng

In [17]:
programming_languages = ['Python', 'Java', 'C++', 'JavaScript', 'Ruby', 'Swift', 'Go', 'PHP', 'C#', 'TypeScript', 'R', 'Kotlin', 'Scala', 'Perl', 'HTML', 'CSS']

language_counts = {language: 0 for language in programming_languages}

for content in tuyendung['content']:
    words = re.findall(r'\b\w+\b', content)
    words = [word.lower() for word in words]
    for language in programming_languages:
        if language.lower() in words:
            language_counts[language] += 1

language_counts_df = pd.DataFrame.from_dict(language_counts, orient='index', columns=['Count'])

language_counts_df = language_counts_df.sort_values(by='Count', ascending=False)

plt.figure(figsize=(10, 6))
language_counts_df.plot(kind='bar', color='skyblue')
plt.title('Programming Languages Usage')
plt.xlabel('Programming Language')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

TypeError: expected string or bytes-like object

# Tạo thuộc tính mới từ text -> Vị trí làm việc, email , ngôn ngữ yêu cầu

### Location 

In [18]:
def read_locations_file(locations_file):
    with open(locations_file, 'r', encoding='utf-8') as file:
        locations = [line.strip() for line in file]
    return locations

def add_location_to_csv(input_csv, locations_file, output_csv):
    locations = read_locations_file(locations_file)
    output_data = []

    with open(input_csv, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            content = row['content'].lower()  
            location = find_location(content, locations)
            row['location'] = location
            output_data.append(row)

    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['post_id', 'content', 'location', 'images']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in output_data:
            writer.writerow(row)

def find_location(content, locations):
    for location in locations:
        if location.lower() in content:
            return location
    return 'Unknown'


### Email

In [31]:
def extract_email(text):
    pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b'
    emails = re.findall(pattern, text)
    return emails[0] if emails else 'Unknown'

def update_analyzed_data_with_email(input_csv_file, output_csv_file):
    with open(input_csv_file, 'r', encoding='utf-8') as input_file:
        reader = csv.DictReader(input_file)
        rows = list(reader)

    for row in rows:
        row['email'] = extract_email(row['content'])

    with open(output_csv_file, 'w', newline='', encoding='utf-8') as output_file:
        fieldnames = ['post_id', 'content', 'images', 'email', 'location']
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

# Ngôn ngữ lập trình

In [40]:
def addProgrammingLanguagesToCSV(input_csv, programming_languages_file, output_csv):
    with open(input_csv, 'r', newline='', encoding='utf-8') as input_file:
        reader = csv.DictReader(input_file)
        rows = list(reader)

    with open(programming_languages_file, 'r', encoding='utf-8') as lang_file:
        programming_languages = [line.strip() for line in lang_file]

    fieldnames = reader.fieldnames + programming_languages

    for row in rows:
        content = row['content'].lower()
        for lang in programming_languages:
            if re.search(r'\b{}\b'.format(re.escape(lang)), content):
                row[lang] = '1'
            else:
                row[lang] = '0'

    with open(output_csv, 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)


In [35]:
input_csv = 'output1.csv'  
locations_file = './Vietnamese-data/Vietnam GIS data/Vnlocations.txt' 
programming_languages_file = './Vietnamese-data/ProgramLanguages.txt'
output_csv = 'analyzed_data.csv'  


In [41]:
add_location_to_csv(input_csv, locations_file, output_csv)
update_analyzed_data_with_email(output_csv, output_csv)
addProgrammingLanguagesToCSV(output_csv, programming_languages_file, output_csv)

In [ ]:
company_name =' \[(?!HN\]|HCM\])(.*?)\]|.*CÔNG TY TNHH.*|(.*?)(?=TUYỂN DỤNG|tuyển dụng|recruit)'

In [ ]:
# matches = re.findall(company_name , tuyendung[0].)
# matches

NameError: name 're' is not defined

In [ ]:
salary =  r'Mức lương:\s*(.*)'

In [ ]:
matches = re.findall(salary, recruitment1)
matches

['14 Triệu - 16 Triệu/tháng']

In [ ]:
position = 'F'